In [144]:
import json
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, select, MetaData, Table
import requests
import sqlalchemy as sa
import urllib
import sql_scripts as ss
from datetime import date, datetime, timedelta
from threading import Thread

In [145]:
def getAuthforWMS(f_data):
    wmsAccess = f_data["wmsAccess"][0]
    return wmsAccess

In [146]:
#gets connections
def getConnforMYSQL(f_data, accessType):
    dialect = pyodbc.drivers()[-1]
    server = f_data[accessType][0]["server"]
    db = f_data[accessType][0]["database"]
    uid = f_data[accessType][0]["uid"]
    pwd = f_data[accessType][0]["pwd"]
    driver = f_data[accessType][0]["dialect_driver"]
    port = f_data[accessType][0]["port"]

    if accessType == "azureAccess":
        connection_string = (
            " Driver={%s}" %dialect +
            "; SERVER=%s" %server + 
            "; Database=%s " %db + 
            "; UID=%s" %uid +
            "; PWD=%s" %pwd
        )
        quoted = urllib.parse.quote_plus(connection_string)
        quoted = f_data[accessType][0]["dialect_driver"] + quoted
        engine = create_engine(quoted, fast_executemany=True).execution_options(isolation_level="AUTOCOMMIT")
    else:
        quoted = driver + uid + ":" + pwd + "@" + server + ":" + str(port) + "/" + db
        engine = create_engine(quoted).execution_options(isolation_level="AUTOCOMMIT")
        
    return engine

In [147]:
#Extracts data from DB
def extractionFunction(conn_integrator, sql_text):
    with conn_integrator.begin():
        str_sql = sa.text(sql_text)
        results = conn_integrator.execute(str_sql)
        columns = results.keys()

        list_columns = []
        list_rows = []
        #print column_names
        for column_name in columns:
            list_columns += [column_name]
        for row in results:
            list_rows += [row]

    return list_rows, list_columns

In [148]:
#get response from API
def setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers, agrs = list):
    #adds default headers
    headers['Accept'] =  "application/json"
    headers['Content-Type'] =  "application/json"   

    completeHTTP = schemeHTTP + baseHTTP + extraHTTP
    response = requests.get(completeHTTP, headers=headers)
    
    return response

In [149]:
def threadMainTask(index, columns_accountList, rows_accountList):
    #begin a MainThread
    print(f'Starting Thread index: {index}')
    
    for column_index in range(len(columns_accountList)):
        if columns_accountList[column_index] == "CONTA":
            account = rows_accountList[index][column_index]
        if columns_accountList[column_index] == "CHAVE":
            key = rows_accountList[index][column_index]
        if columns_accountList[column_index] == "TOKEN":
            token = rows_accountList[index][column_index]



    now = datetime.now().strftime(ss.queryList['getAllNewOrdersVtex_Recent']['nowStrftime'])
    yesterday = (datetime.now() - timedelta(days=ss.queryList['getAllNewOrdersVtex_Recent']['timedelta'])).strftime(ss.queryList['getAllNewOrdersVtex_Recent']['yesterdayStrftime'])
    
    schemeHTTP = "https://"
    baseHTTP = "%s.%s.com.br" % (account, "vtexcommercestable")
    headers = {                
            'X-VTEX-API-AppKey': key,
            'X-VTEX-API-AppToken': token
            }
    pageNumber = 1
    extraHTTP = "/api/oms/pvt/orders?page=%s&f_creationDate=creationDate:[%s TO %s]" %(pageNumber, yesterday, now)

    #get first response
    response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers, [])

    print("response status_code: ", response.status_code)
    #good Responses
    if response.status_code == 200:
        #include initial dataDict
        globals()["dataList"] = globals()["dataList"] + response.json()["list"]

        subThreads = []

        #get paging info
        for pageNumber in range(response.json()["paging"]["pages"]):
            #call subThread
            if pageNumber > 1:
                print(f'Starting subThread index: {index} pageNumber: {pageNumber}')
                st = Thread(target=threadSubTask, args=(pageNumber, schemeHTTP, baseHTTP, headers, yesterday, now,))
                subThreads.append(st)
                st.start()
        
        for st in subThreads:
            st.join()
    #Bad Responses
    else:
        globals()["errorList"].append([{'account': account, 'status_code': response.status_code, 'time': now }])

In [150]:
def threadSubTask(pageNumber, schemeHTTP, baseHTTP, headers, yesterday, now):
    extraHTTP = "/api/oms/pvt/orders?page=%s&f_creationDate=creationDate:[%s TO %s]" %(pageNumber, yesterday, now)
    response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers, [])
    globals()["dataList"] = globals()["dataList"] + response.json()["list"]

In [151]:
def main ():
    #open auth file
    auth = open('auth.json')
    auth_load = json.load(auth)

    #getting conn for azure
    engine_azure = getConnforMYSQL(auth_load, "azureAccess")
    conn_azure = engine_azure.connect()

    #getData from azureDB about vTex Account list
    rows_accountList, columns_accountList = extractionFunction(conn_azure, ss.queryList['getAllNewOrdersVtex_Recent']['sql_query'] )

    #list of main threads
    threads = []
    
    #start the dataList
    globals()["dataList"] = []

    #start the errorList
    globals()["errorList"] = []

    for index in range(len(rows_accountList)):
        t = Thread(target=threadMainTask, args=(index, columns_accountList, rows_accountList, ))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()

    #treat invoiceOutput 
    for d in globals()['dataList']:
        if d['invoiceOutput'] != None:
            d['invoiceOutput'] = d['invoiceOutput'][0]
    
    df = pd.DataFrame.from_dict(globals()["dataList"])
    df2 = pd.DataFrame.from_dict(globals()["errorList"])
    
    #columnsDateList = ['creationDate','ShippingEstimatedDate','ShippingEstimatedDateMax','ShippingEstimatedDateMin','authorizedDate','lastChange', 'paymentApprovedDate','readyForHandlingDate']
    #for column in columnsDateList:
    #    df[[column]]= df[[column]].apply(pd.to_datetime)

    #fillNan with empty string
    df = df.fillna("")
    df2 = df2.fillna("")
    
    df.to_sql("temp_OrdersVtexSuccess", engine_azure, if_exists='replace', index=False)
    if len(globals()["errorList"]) > 0:
        df2.to_sql("temp_OrdersVtexfailure", engine_azure, if_exists='append', index=False)


    conn_azure.close()
    

In [152]:
if __name__ == "__main__":
    main()
    print('getOrdersVtex: done')

Starting Thread index: 0Starting Thread index: 1

Starting Thread index: 2
Starting Thread index: 3
Starting Thread index: 4
Starting Thread index: 5
Starting Thread index: 6
Starting Thread index: 7
Starting Thread index: 8
Starting Thread index: 9
Starting Thread index: 10
Starting Thread index: 11
Starting Thread index: 12
Starting Thread index: 13
Starting Thread index: 14
Starting Thread index: 15
Starting Thread index: 16
Starting Thread index: 17
response status_code: response status_code:  402
 200
response status_code:  200
response status_code:  402
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
Starting subThread index: 5 pageNumber: 2
Starting subThread index: 5 pageNumber: 3
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200
response status_code:  200